- Multilevel Ordinal Logistic regression을 활용해서 StateName, BreedName 등 군집성이 높은 컬럼들을 바탕으로 ordinal classification 진행

## Import Data

In [ ]:
install.packages('ordinal')
library(ordinal)
library(nlme)
library(package = feather)
library(pscl)
library(MASS)

In [34]:
df <- read_feather('merged_data.feather')

Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”

## Data Preprocessing

EDA를 통해 파악한 컬럼 특성을 바탕으로 데이터 전처리 진행

In [36]:
# Description을 제외하고 필터링
df_trim = dplyr::select(df, -Description, -PetID, -RescuerID, -Name)
        
# is_train이 True인 데이터 열만 추림
df_trim = dplyr::filter(df_trim, is_train)
        
# 각각의 컬럼을 categorical 형으로 변환
df_trim$Dewormed = as.factor(df_trim$Dewormed)
df_trim$Gender = as.factor(df_trim$Gender)
df_trim$Vaccinated = as.factor(df_trim$Vaccinated)
df_trim$Sterilized = as.factor(df_trim$Sterilized)
df_trim$Type = as.factor(df_trim$Type)
df_trim$VideoAmt = as.factor(df_trim$VideoAmt)
df_trim$ColorName1 = as.factor(df_trim$ColorName1)
df_trim$ColorName2 = as.factor(df_trim$ColorName2)
df_trim$ColorName3 = as.factor(df_trim$ColorName3)
df_trim$BreedName1 = as.factor(df_trim$BreedName1)
df_trim$BreedName2 = as.factor(df_trim$BreedName2)
df_trim$StateName = as.factor(df_trim$StateName)
df_trim$AdoptionSpeed <- as.ordered(df_trim$AdoptionSpeed)

## Modeling

### 1) Baseline 모델

과거 EDA 과정에서 보여졌듯이 StateName 별 입양 특성 차이가 극명하게 나는 것을 확인함.이 지식을 바탕으로 StateName을 베이스로하는 모델을 제작하고 해석해보자.

In [64]:
model1 <-clmm(AdoptionSpeed~ 1+(1|StateName), data=df_trim)

In [65]:
summary(model1)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: AdoptionSpeed ~ 1 + (1 | StateName)
data:    df_trim

 link  threshold nobs  logLik    AIC      niter    max.grad cond.H 
 logit flexible  14993 -21905.65 43821.29 258(596) 3.83e-03 7.9e+02

Random effects:
 Groups    Name        Variance Std.Dev.
 StateName (Intercept) 0.1257   0.3545  
Number of groups:  StateName 14 

No Coefficients

Threshold coefficients:
    Estimate Std. Error z value
0|1  -3.8369     0.1243 -30.870
1|2  -1.4494     0.1153 -12.569
2|3  -0.2427     0.1147  -2.116
3|4   0.6991     0.1148   6.090

- Std.Dev가 0.3545으로 0 보다 크다는 것은 StateName 별의 AdoptionSpeed 분산이 존재함을 뜻함
- AIC: 43821.29

### 2) StateName 별 Type이 AdoptionSpeed에 미치는 영향을 고려한 모델

In [84]:
model2.1 <-clmm(AdoptionSpeed~ Type +(1|StateName), data=df_trim)

Warning message in update.uC(rho):
“step factor reduced below minimum when updating the random effects
  at iteration 526”Warning message in update.uC(rho):
“iteration limit reached when updating the random effects
  at iteration 534”Warning message in update.uC(rho):
“step factor reduced below minimum when updating the random effects
  at iteration 620”

In [85]:
summary(model2.1)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: AdoptionSpeed ~ Type + (1 | StateName)
data:    df_trim

 link  threshold nobs  logLik    AIC      niter    max.grad cond.H 
 logit flexible  14993 -21849.45 43710.90 361(939) 2.78e-02 8.2e+02

Random effects:
 Groups    Name        Variance Std.Dev.
 StateName (Intercept) 0.1167   0.3416  
Number of groups:  StateName 14 

Coefficients:
        Estimate Std. Error z value Pr(>|z|)    
TypeDog  0.31645    0.02989   10.59   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Threshold coefficients:
    Estimate Std. Error z value
0|1 -3.68627    0.12262 -30.064
1|2 -1.29181    0.11356 -11.376
2|3 -0.07722    0.11307  -0.683
3|4  0.86895    0.11321   7.675

- StateName별 TypeDog가 AdoptionSpeed에 미치는 영향이 통계적으로 유의미함을 보임. 따라서 Type이 Dog일 경우에 입양이 상대적으로 늦게됨.
- Model#1 대비 AIC가 비교적 감소함 (43710.9): 좀더 개선된 모델임을 알 수 있음.
- StateName의 Std.Dev.는 감소 → Type을 감안하게 될 경우 군집별 (StateName) 분산이 줄어드는 것을 보임. 그 뜻은 많은 State들에서 Cat을 선호하기에 Type을 감안하게 되면 전반적으로 군집별 차이가 다소 감소함을 뜻함.

## Model 비교

과연 Multilevel (StateName)을 고려하지 않은 모델과 Model#2는 어떤 차이를 가질까? \
여러 Ordinal logistic regression 들과 비교해보자.

#### 1) AdoptionSpeed~ Type

In [94]:
model2.2 <- polr(AdoptionSpeed~ Type, data=df_trim, Hess = TRUE)
summary(model2.2)

Call:
polr(formula = AdoptionSpeed ~ Type, data = df_trim, Hess = TRUE)

Coefficients:
         Value Std. Error t value
TypeDog 0.3243    0.02945   11.01

Intercepts:
    Value    Std. Error t value 
0|1  -3.4074   0.0521   -65.3622
1|2  -1.0175   0.0247   -41.2282
2|3   0.1909   0.0231     8.2497
3|4   1.1295   0.0249    45.4042

Residual Deviance: 43819.10 
AIC: 43829.10 

- 위 모델의 결과와 Model#2를 비교해보면 TypeDog의 coefficient 값이 살짝 차이나는 것을 볼 수 있음.
- 두 모델 모두 TypeDog이 통계적으로 유의미함을 보임.
- 하지만 AIC의 경우 Model#2가 더 개선됨.
- 따라서 위 모델에서 설명되지 못하는 StateName별 차이를 Multilevel modeling의 경우 더 잘 설명함을 알 수 있음.

#### 2) AdoptionSpeed ~ Gender + Type

In [113]:
model3.1 <- polr(AdoptionSpeed ~ Gender + Type, data=df_trim, Hess = TRUE)
summary(model3.1)

Call:
polr(formula = AdoptionSpeed ~ Gender + Type, data = df_trim, 
    Hess = TRUE)

Coefficients:
               Value Std. Error t value
GenderMale  -0.21884    0.03195  -6.849
GenderMixed  0.08231    0.04416   1.864
TypeDog      0.33008    0.02968  11.120

Intercepts:
    Value    Std. Error t value 
0|1  -3.4798   0.0549   -63.3861
1|2  -1.0871   0.0299   -36.3353
2|3   0.1256   0.0284     4.4181
3|4   1.0674   0.0297    35.9144

Residual Deviance: 43754.12 
AIC: 43768.12 

In [114]:
model3.2 <-clmm(AdoptionSpeed ~ Gender + Type + (1|StateName), data=df_trim)
summary(model3.2)

Warning message in update.uC(rho):
“step factor reduced below minimum when updating the random effects
  at iteration 774”Warning message in update.uC(rho):
“iteration limit reached when updating the random effects
  at iteration 792”Warning message in update.uC(rho):
“iteration limit reached when updating the random effects
  at iteration 862”Warning message in update.uC(rho):
“step factor reduced below minimum when updating the random effects
  at iteration 926”Warning message in update.uC(rho):
“step factor reduced below minimum when updating the random effects
  at iteration 949”Warning message in update.uC(rho):
“step factor reduced below minimum when updating the random effects
  at iteration 1005”Warning message in update.uC(rho):
“iteration limit reached when updating the random effects
  at iteration 1019”Warning message in update.uC(rho):
“step factor reduced below minimum when updating the random effects
  at iteration 1348”Warning message in update.uC(rho):
“step factor red

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: AdoptionSpeed ~ Gender + Type + (1 | StateName)
data:    df_trim

 link  threshold nobs  logLik    AIC      niter     max.grad cond.H 
 logit flexible  14993 -21818.78 43653.56 735(1912) 7.74e-03 8.2e+02

Random effects:
 Groups    Name        Variance Std.Dev.
 StateName (Intercept) 0.1153   0.3396  
Number of groups:  StateName 14 

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
GenderMale  -0.21191    0.03201  -6.621 3.58e-11 ***
GenderMixed  0.08290    0.04425   1.873    0.061 .  
TypeDog      0.32297    0.03011  10.728  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Threshold coefficients:
    Estimate Std. Error z value
0|1  -3.7512     0.1224 -30.648
1|2  -1.3541     0.1134 -11.941
2|3  -0.1355     0.1129  -1.201
3|4   0.8137     0.1130   7.202

- 1) AdoptionSpeed~ Type 의 결과와 유사한 양상을 보임.
- 두 모델 모두에서 GenderMale, TypeDog가 통계적으로 유의미함을 보임.
- StateName별 차이가 분명 존재하며 개선된 AIC로 비교 가능.

#### 3) AdoptionSpeed ~ Gender + Type + Gender * Type (교호작용 고려)

In [115]:
model4.1 <- polr(AdoptionSpeed ~ Gender + Type + Gender * Type, data=df_trim, Hess = TRUE)
summary(model4.1)

Call:
polr(formula = AdoptionSpeed ~ Gender + Type + Gender * Type, 
    data = df_trim, Hess = TRUE)

Coefficients:
                       Value Std. Error t value
GenderMale          -0.13290    0.04857 -2.7361
GenderMixed          0.08157    0.06004  1.3587
TypeDog              0.38375    0.04275  8.9776
GenderMale:TypeDog  -0.15250    0.06444 -2.3666
GenderMixed:TypeDog  0.02575    0.08916  0.2888

Intercepts:
    Value    Std. Error t value 
0|1  -3.4483   0.0578   -59.6621
1|2  -1.0555   0.0350   -30.1634
2|3   0.1575   0.0338     4.6591
3|4   1.0997   0.0350    31.4423

Residual Deviance: 43747.32 
AIC: 43765.32 

In [116]:
model4.2 <-clmm(AdoptionSpeed ~ Gender + Type + Gender * Type + (1|StateName), data=df_trim)
summary(model4.2)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: AdoptionSpeed ~ Gender + Type + Gender * Type + (1 | StateName)
data:    df_trim

 link  threshold nobs  logLik    AIC      niter      max.grad cond.H 
 logit flexible  14993 -21815.52 43651.04 1238(2649) 2.99e-02 8.3e+02

Random effects:
 Groups    Name        Variance Std.Dev.
 StateName (Intercept) 0.1143   0.338   
Number of groups:  StateName 14 

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
GenderMale          -0.12659    0.04864  -2.603  0.00925 ** 
GenderMixed          0.08505    0.06013   1.415  0.15721    
TypeDog              0.37733    0.04311   8.754  < 2e-16 ***
GenderMale:TypeDog  -0.15150    0.06456  -2.346  0.01895 *  
GenderMixed:TypeDog  0.01883    0.08926   0.211  0.83293    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Threshold coefficients:
    Estimate Std. Error z value
0|1  -3.7188     0.1240 -29.986
1|2  -1.3216     0.1151 -11.484
2

- 교호작용 (GenderMale:TypeDog)이 통계적 유의미함을 보이며 수컷 강아지가 입양이 더 빨리되는 것을 보임.
- Multilevel 모델 에서 더 개선된 AIC가 나타남으로 StateName간 차이가 존재함을 나타냄.

#### Multilevel (BreedName)을 고려한 모델과 고려하지 않은 Ordinal logistic regression를 비교해보자.

#### 4) AdoptionSpeed ~ Age + Health

In [117]:
model5.1 <-polr(AdoptionSpeed ~ Age + Health, data=df_trim, Hess = TRUE)
summary(model5.1)

Call:
polr(formula = AdoptionSpeed ~ Age + Health, data = df_trim, 
    Hess = TRUE)

Coefficients:
         Value Std. Error t value
Age    0.01033  0.0008496  12.160
Health 0.18845  0.0753091   2.502

Intercepts:
    Value    Std. Error t value 
0|1  -3.2852   0.0920   -35.7159
1|2  -0.8991   0.0796   -11.2899
2|3   0.3086   0.0792     3.8961
3|4   1.2523   0.0799    15.6688

Residual Deviance: 43770.58 
AIC: 43782.58 

In [118]:
model5.2 <-clmm(AdoptionSpeed ~ Age + Health + (1|BreedName1), data=df_trim)
summary(model5.2)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: AdoptionSpeed ~ Age + Health + (1 | BreedName1)
data:    df_trim

 link  threshold nobs  logLik    AIC      niter     max.grad cond.H 
 logit flexible  14976 -21466.38 42946.76 350(1059) 4.12e-02 5.1e+04

Random effects:
 Groups     Name        Variance Std.Dev.
 BreedName1 (Intercept) 0.3679   0.6066  
Number of groups:  BreedName1 174 

Coefficients:
       Estimate Std. Error z value Pr(>|z|)    
Age    0.020658   0.001049  19.695  < 2e-16 ***
Health 0.225538   0.076279   2.957  0.00311 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Threshold coefficients:
    Estimate Std. Error z value
0|1  -2.8441     0.1168 -24.342
1|2  -0.3968     0.1077  -3.686
2|3   0.8633     0.1077   8.019
3|4   1.8462     0.1085  17.023
(17 observations deleted due to missingness)

- model5.1과 model5.2를 비교했을때 확연한 차이가 나타남.
- model5.2 (Multilevel)에서 Std.Dev 0.6으로 앞의 multilevel 모델들에 비해서 월등히 큼. 이는 BreedName 별로 차이가 난다는 것을 뜻함.
- 두 모델 모두 Age와 Health가 통계적으로 유의미함을 보이며 나이가 많을수록, 건강이 나쁠수록 입양이 늦게되며 특히나 Health가 더 큰 영향을 끼침.
- model5.2가 훨씬 더 더 개선된 AIC가 나타남으로 BreedName간 차이가 존재함을 알 수 있음.